# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, normal
import os
import time 

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset

url_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
dataset = TabularDatasetFactory.from_delimited_files(path = url_path)



In [4]:
from azureml.core import Experiment, Workspace

ws = Workspace.from_config()
experiment_name = 'hyperparameter_tuning'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [10]:
from azureml.train.hyperdrive.policy import MedianStoppingPolicy


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = MedianStoppingPolicy(delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(1,10,100),
        "--max_iter": choice(50, 100, 150)})

amlcompute_cluster_name = 'Gasto'
vm_size = 'Standard_D2_V2'

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target=amlcompute_cluster_name, 
                vm_size=vm_size, vm_priority='lowpriority', entry_script='train2.py')

hyperdrive_run_config = HyperDriveConfig(run_config=None,
                             estimator=estimator,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [11]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperparameter_tuning,HD_69e75294-522c-4120-a383-dea8e8fac504,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
%%time

run_status = hyperdrive_run.wait_for_completion()
print(run_status['status'])
if run_status['status'] != 'Completed':
    raise Exception('The run did not successfully complete.')

Completed
CPU times: user 896 ms, sys: 74.7 ms, total: 970 ms
Wall time: 10min 16s


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()

print('Best run ID: ', best_run.id)
print('Best run accuracy:', best_run_metrics['Accuracy'])

RunDetails(hyperdrive_run).show()

Best run ID:  HD_69e75294-522c-4120-a383-dea8e8fac504_0
Best run accuracy: 0.7575757575757576


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [17]:
##TODO: Save the best model
os.makedirs('./outputs', exist_ok=True)


best_run.register_model(model_name = 'hyperml_best_model.pkl', model_path = './outputs/')

import joblib

#joblib.dump(value= fitted_model, filename = 'outputs/hyper_ML_fitted.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
#model = best_run.register_model(model_name='model_name', model_path='./outputs/model_name.pkl',model_framework=Model.Framework.SCIKITLEARN)

model = best_run.register_model(model_name = 'hyperml_best_model.pkl', model_path = './outputs/')
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                                enable_app_insights=True, auth_enabled=True)  
service_name = 'my-hypertune-service'
service = Model.deploy(ws, service_name, [model], deployment_config=aci_config)
service.wait_for_deployment(show_output = True)
print(service.get_logs())

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model_name.pkl in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_69796bc39840e8c5ea315847eeccdf0195983874aaee24aa05f6057ec7fba489_d.txt', 'azureml-logs/65_job_prep-tvmps_69796bc39840e8c5ea315847eeccdf0195983874aaee24aa05f6057ec7fba489_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_69796bc39840e8c5ea315847eeccdf0195983874aaee24aa05f6057ec7fba489_d.txt', 'logs/azureml/102_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model_logreg.joblib']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/model_name.pkl in the set of files upl

TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
import requests
import json
# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
#key = ''

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
           "age": 75, 
           "anaemia": 0, 
           "creatinine_phosphokinase": 582, 
           "diabetes": 0, 
           "ejection_fraction": 20, 
           "high_blood_pressure": 1, 
           "platelets": 265000, 
           "serum_creatinine": 1.9, 
           "serum_sodium": 130, 
           "sex": 1, 
           "smoking": 0,
           "time": 4
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)
# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'
# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

NameError: name 'service' is not defined

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()